## **Import Libraries**

In [1]:
# import libraries

import pandas as pd
import torch                                 # pytorch
from torch_geometric.data import Data        # to work with graph data

## **Import Dataset**

In [2]:
# import csv 

sensor_df_raw = pd.read_csv(r"C:\Users\colin\OneDrive\Desktop\Thesis Part 2\thesis - simulation\data\sensor.csv")
buildings_df_raw = pd.read_csv(r"C:\Users\colin\OneDrive\Desktop\Thesis Part 2\thesis - simulation\data\buildings.csv")
labels_df_raw = pd.read_csv(r"C:\Users\colin\OneDrive\Desktop\Thesis Part 2\thesis - simulation\data\labels.csv")
distance_df = pd.read_csv(r"C:\Users\colin\OneDrive\Desktop\Thesis Part 2\thesis - simulation\data\distance.csv")
sensor_length_df = pd.read_csv(r"C:\Users\colin\OneDrive\Desktop\Thesis Part 2\thesis - simulation\data\sensor_length.csv")
vertex_length_df = pd.read_csv(r"C:\Users\colin\OneDrive\Desktop\Thesis Part 2\thesis - simulation\data\vertex_length.csv")
print('imported')

imported


## **Prepare Dataset - sensor, building, labels.csv**

In [3]:
# transpose the data

sensor_df= sensor_df_raw.transpose()
buildings_df= buildings_df_raw.transpose()
labels_df= labels_df_raw.transpose()

In [4]:
# function to drop first row (integer indices)

def drop_indice(df):
    df.columns = df.iloc[0]   # Set the first row as column names
    df = df.iloc[1:]  # Remove the first row
    df.reset_index(drop=True, inplace=True)  # Reset the index
    return df

In [5]:
# drop indices

sensor_df = drop_indice(sensor_df)
buildings_df = drop_indice(buildings_df)
labels_df = drop_indice(labels_df)

In [6]:
sensor_df.head()

sensor_id,sensor_x_coordinate,sensor_y_coordinate,sensor_z_coordinate
0,360.273651,438.728317,1.2
1,360.273651,438.728317,3.6
2,360.273651,438.728317,6.0
3,360.273651,438.728317,8.4
4,360.273651,438.728317,10.8


In [7]:
buildings_df.head()

vertex_id,x_coordinate,y_coordinate,z_coordinate
0,349.811102,472.454332,0.0
1,350.259085,473.223226,0.0
2,350.259085,473.223226,45.0
3,349.811102,472.454332,45.0
4,360.19802,467.538691,0.0


In [8]:
labels_df.head()

sensor_id,hb_solar_radiation
0,861.793137
1,857.606391
2,856.760583
3,854.764257
4,854.645182


## **Prepare Dataset - edge.csv**

In [9]:
distance_df = pd.read_csv(r"C:\Users\colin\OneDrive\Desktop\Thesis Part 2\thesis - simulation\data\distance.csv")
sensor_length_df = pd.read_csv(r"C:\Users\colin\OneDrive\Desktop\Thesis Part 2\thesis - simulation\data\sensor_length.csv")
vertex_length_df = pd.read_csv(r"C:\Users\colin\OneDrive\Desktop\Thesis Part 2\thesis - simulation\data\vertex_length.csv")
print('successful')

successful


In [10]:
#check
distance_df.head()

,distance
0,35.331982
1,35.939258
2,56.644773
3,56.261434
4,28.835453


In [11]:
distance_df.shape

(117780, 1)

In [12]:
#check
sensor_length_df.head()

,453


In [13]:
#check
vertex_length_df.head()

,260


In [14]:
#extract out the length values from both dataframes

sensor_length = sensor_length_df.columns[0]
print(sensor_length)
vertex_length = vertex_length_df.columns[0]
print(vertex_length)

453
260


In [15]:
# convert length to numerical from string values 

sensor_length = int(sensor_length)
print(type(sensor_length))

vertex_length = int(vertex_length)
print(type(vertex_length))

<class 'int'>
<class 'int'>


In [16]:
def create_temp_df(n, m):
    # n is the number of sensors, m is the number of vertices
    data = []  # List to hold the data
    for sensor_length in range(1, n+1):  # Iterate through each sensor
        for vertex_length in range(1, m + 1):  # Iterate through each vertex for the sensor
            # Append a tuple with the sensor ID and vertex ID to the list
            data.append((f'sensor_id_{sensor_length}', f'vertex_{vertex_length}'))
    
    # Create DataFrame from the data list
    temp_df = pd.DataFrame(data, columns=['sensor_id', 'vertex_id'])
    
    return temp_df

In [17]:
# call function to create sensor id and vertex id dataframe

temp_df = create_temp_df(sensor_length, vertex_length)
temp_df

,sensor_id,vertex_id
0,sensor_id_1,vertex_1
1,sensor_id_1,vertex_2
2,sensor_id_1,vertex_3
3,sensor_id_1,vertex_4
4,sensor_id_1,vertex_5
...,...,...
117775,sensor_id_453,vertex_256
117776,sensor_id_453,vertex_257
117777,sensor_id_453,vertex_258
117778,sensor_id_453,vertex_259


In [18]:
temp_df['distance'] = distance_df['distance']
edge_df = temp_df

In [38]:
edge_df

,sensor_id,vertex_id,distance
0,sensor_id_1,vertex_1,35.331982
1,sensor_id_1,vertex_2,35.939258
2,sensor_id_1,vertex_3,56.644773
3,sensor_id_1,vertex_4,56.261434
4,sensor_id_1,vertex_5,28.835453
...,...,...,...
117775,sensor_id_453,vertex_256,45.543195
117776,sensor_id_453,vertex_257,56.887285
117777,sensor_id_453,vertex_258,46.226373
117778,sensor_id_453,vertex_259,54.249894


## **Prepare Dataset - Null Check**

In [19]:
# function for null values check

def null_check(df):
    total_rows = len(df)
    for column in df.columns:
        null_count = df[column].isnull().sum()
        null_percentage = round((null_count/total_rows)*100,2)
        print('_'*20)
        print(f'{column}')
        print('_'*20)
        print(f'Null count: {null_count}')
        print(f'Percentage : {null_percentage}')

In [20]:
null_check(sensor_df)
null_check(buildings_df)
null_check(labels_df)
null_check(distance_df)

____________________
sensor_x_coordinate
____________________
Null count: 0
Percentage : 0.0
____________________
sensor_y_coordinate
____________________
Null count: 0
Percentage : 0.0
____________________
sensor_z_coordinate
____________________
Null count: 0
Percentage : 0.0
____________________
x_coordinate
____________________
Null count: 0
Percentage : 0.0
____________________
y_coordinate
____________________
Null count: 0
Percentage : 0.0
____________________
z_coordinate
____________________
Null count: 0
Percentage : 0.0
____________________
hb_solar_radiation
____________________
Null count: 0
Percentage : 0.0
____________________
distance
____________________
Null count: 0
Percentage : 0.0


## **Preprocess Node Index and Naming**

In [21]:
# Resetting the index so 'sensor_id' is no longer the index column
sensor_df.reset_index(drop=True, inplace=True)

# Creating a new 'sensor_id' column with formatted values
sensor_df['sensor_id'] = ['sensor_id_' + str(i + 1) for i in sensor_df.index]

# Insert the 'sensor_id_formatted' column as the first column
sensor_df.insert(0, 'sensor_id', sensor_df.pop('sensor_id'))

In [22]:
# Resetting the index so 'sensor_id' is no longer the index column
buildings_df.reset_index(drop=True, inplace=True)

# Creating a new 'sensor_id' column with formatted values
buildings_df['vertex_id'] = ['vertex_id_' + str(i + 1) for i in buildings_df.index]

# Insert the 'sensor_id_formatted' column as the first column
buildings_df.insert(0, 'vertex_id', buildings_df.pop('vertex_id'))

In [23]:
# Resetting the index so 'sensor_id' is no longer the index column
labels_df.reset_index(drop=True, inplace=True)

# Creating a new 'sensor_id' column with formatted values
labels_df['sensor_id'] = ['sensor_id_' + str(i + 1) for i in labels_df.index]

# Insert the 'sensor_id_formatted' column as the first column
labels_df.insert(0, 'sensor_id', labels_df.pop('sensor_id'))

## **Final Check**

In [24]:
print('sensor_df')
print(sensor_df.head())
print('buildings_df')
print(buildings_df.head())
print('labels_df')
print(labels_df.head())
print('edge_df')
print(edge_df.head())

sensor_df
sensor_id    sensor_id sensor_x_coordinate sensor_y_coordinate  \
0          sensor_id_1          360.273651          438.728317   
1          sensor_id_2          360.273651          438.728317   
2          sensor_id_3          360.273651          438.728317   
3          sensor_id_4          360.273651          438.728317   
4          sensor_id_5          360.273651          438.728317   

sensor_id sensor_z_coordinate  
0                         1.2  
1                         3.6  
2                         6.0  
3                         8.4  
4                        10.8  
buildings_df
vertex_id    vertex_id x_coordinate y_coordinate z_coordinate
0          vertex_id_1   349.811102   472.454332          0.0
1          vertex_id_2   350.259085   473.223226          0.0
2          vertex_id_3   350.259085   473.223226         45.0
3          vertex_id_4   349.811102   472.454332         45.0
4          vertex_id_5    360.19802   467.538691          0.0
labels_df
sensor

## **Overall Graph Object**

- Node Features
- Edge Index
- Edge Attributes
- Combine to form data object

## **Prepare Node Features**

Combine sensor and building information to create a unified node feature matrix. 

Combine sensor_df and buildings_df into a single dataframe, ensuring each has a unique identifier across sensors and vertices

In [25]:
# Add a column to distinguish between sensors and vertices
sensor_df['type'] = 'sensor'
buildings_df['type'] = 'vertex'

In [26]:
# Combine dataframes
all_nodes_df = pd.concat([sensor_df.assign(index=range(0, len(sensor_df))),
                          buildings_df.assign(index=range(len(sensor_df), len(sensor_df)+len(buildings_df)))])


In [27]:
all_nodes_df

,sensor_id,sensor_x_coordinate,sensor_y_coordinate,sensor_z_coordinate,type,index,vertex_id,x_coordinate,y_coordinate,z_coordinate
0,sensor_id_1,360.273651,438.728317,1.2,sensor,0,NaN,NaN,NaN,NaN
1,sensor_id_2,360.273651,438.728317,3.6,sensor,1,NaN,NaN,NaN,NaN
2,sensor_id_3,360.273651,438.728317,6.0,sensor,2,NaN,NaN,NaN,NaN
3,sensor_id_4,360.273651,438.728317,8.4,sensor,3,NaN,NaN,NaN,NaN
4,sensor_id_5,360.273651,438.728317,10.8,sensor,4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
255,NaN,NaN,NaN,NaN,vertex,708,vertex_id_256,345.189406,481.453813,45.0
256,NaN,NaN,NaN,NaN,vertex,709,vertex_id_257,344.378254,481.917478,0.0
257,NaN,NaN,NaN,NaN,vertex,710,vertex_id_258,344.378254,481.917478,45.0
258,NaN,NaN,NaN,NaN,vertex,711,vertex_id_259,341.606353,477.146624,0.0


In [28]:
# Prepare node features - example: using coordinates and a type flag (sensor=1, vertex=0)
all_nodes_df['type_flag'] = all_nodes_df['type'].apply(lambda x: 1 if x == 'sensor' else 0)
node_features = all_nodes_df[['sensor_x_coordinate', 'sensor_y_coordinate', 'sensor_z_coordinate', 'type_flag']].fillna(0).values
x = torch.tensor(node_features, dtype=torch.float)

In [29]:
# Print the tensor to check its contents
print("Node Features Tensor:")
print(x)

Node Features Tensor:
tensor([[360.2737, 438.7283,   1.2000,   1.0000],
        [360.2737, 438.7283,   3.6000,   1.0000],
        [360.2737, 438.7283,   6.0000,   1.0000],
        ...,
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000]])


## **Create Edge Index**

1. Map each sensor_id and vertex_id to a unique index.2. 
Use these mappings to create the edge_index tensor from edge_df.

Step 1: Prepare Mappings
First, we'll create dictionaries to map sensor_id and vertex_id to unique indices. We'll concatenate the indices of sensors and vertices to ensure uniqueness across the graph.

In [39]:
sensor_ids = sensor_df['sensor_id'].unique()
vertex_ids = buildings_df['vertex_id'].unique()

# Create a continuous index for sensors and vertices
sensor_index = {sensor_id: i for i, sensor_id in enumerate(sensor_ids)}
vertex_index = {vertex_id: i + len(sensor_index) for i, vertex_id in enumerate(vertex_ids)}

Step 2: Create Edge Index
Next, we'll use these mappings to create the edge_index tensor. Note that the vertex_id in edge_df appears to have a slight discrepancy (missing the "id_" part based on the example provided), so we'll adjust for that in our mapping.

In [40]:
# Adjust the vertex_id in edge_df to match the format in buildings_df
edge_df['adjusted_vertex_id'] = edge_df['vertex_id'].apply(lambda x: 'vertex_id_' + x.split('_')[-1])

# Map sensor_id and vertex_id to their respective indices
edge_index_list = edge_df.apply(lambda row: [sensor_index.get(row['sensor_id'], -1),
                                             vertex_index.get(row['adjusted_vertex_id'], -1)], axis=1)

# Filter out any edges that couldn't be mapped (-1 indicates a mapping failure)
filtered_edge_index_list = [pair for pair in edge_index_list if -1 not in pair]

# Convert to torch tensor
edge_index = torch.tensor(filtered_edge_index_list, dtype=torch.long).t().contiguous()


In [41]:
edge_index

tensor([[  0,   0,   0,  ..., 452, 452, 452],
        [453, 454, 455,  ..., 710, 711, 712]])

## **Edge Attributes**

In [33]:
edge_attr = torch.tensor(edge_df[['distance']].values, dtype=torch.float)

In [42]:
edge_attr

tensor([[35.3320],
        [35.9393],
        [56.6448],
        ...,
        [46.2264],
        [54.2499],
        [42.9391]])

## **Target Labels**

In [43]:
# Ensure data type compatibility
labels_df['hb_solar_radiation'] = labels_df['hb_solar_radiation'].astype(float)

In [44]:
# Update labels for sensors with their radiation values
labels_df['index'] = labels_df['sensor_id'].map(sensor_to_index)

In [46]:
# Create torch tensor with compatible data type
labels = torch.zeros(len(labels_df), dtype=torch.float)
labels[labels_df['index']] = torch.tensor(labels_df['hb_solar_radiation'].values, dtype=torch.float)

## **Create a PyTorch Geometric Data Object**

In [47]:
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=labels)

In [48]:
# Print check for the Data object
print("Data Object:")
print(data)

Data Object:
Data(x=[713, 4], edge_index=[2, 117780], edge_attr=[117780, 1], y=[453])
